In [29]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
sns.set_palette('colorblind')

In [30]:
df = pd.read_csv('data/COVID-19_Outcomes_by_Testing_Cohorts__Cases__Hospitalizations__and_Deaths.csv')
df.head()

,extract_date,specimen_date,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
0,04/29/2020,04/17/2020,9979,3386,527,96
1,04/29/2020,02/08/2020,1,0,0,0
2,04/29/2020,03/05/2020,63,5,3,1
3,04/29/2020,04/09/2020,9019,4803,1253,386
4,04/29/2020,04/03/2020,9389,5523,1688,582


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176148 entries, 0 to 176147
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   extract_date         176148 non-null  object
 1   specimen_date        176148 non-null  object
 2   Number_tested        176148 non-null  int64 
 3   Number_confirmed     176148 non-null  int64 
 4   Number_hospitalized  176148 non-null  int64 
 5   Number_deaths        176148 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 8.1+ MB


In [32]:
df.describe()

,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
count,176148.000000,176148.000000,176148.000000,176148.000000
mean,11141.475714,1427.914680,234.946375,73.940686
std,6413.050113,1368.222368,392.006729,160.110002
min,1.000000,0.000000,0.000000,0.000000
25%,6738.000000,371.000000,38.000000,4.000000
50%,10885.000000,1135.000000,115.000000,21.000000
75%,16448.000000,1960.000000,199.000000,44.000000
max,24727.000000,6852.000000,2016.000000,923.000000


In [33]:
valid = df[df['Number_tested'] > 1]

In [34]:
valid['specimen_date'] = pd.to_datetime(valid['specimen_date'])

/tmp/ipykernel_5442/2117046692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['specimen_date'] = pd.to_datetime(valid['specimen_date'])


In [35]:
good_dates = valid[valid['specimen_date'] >= '1/1/2020']

In [36]:
good_dates = valid[valid['specimen_date'] <= '10/12/2021']

In [37]:
good_dates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170140 entries, 0 to 176147
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   extract_date         170140 non-null  object        
 1   specimen_date        170140 non-null  datetime64[ns]
 2   Number_tested        170140 non-null  int64         
 3   Number_confirmed     170140 non-null  int64         
 4   Number_hospitalized  170140 non-null  int64         
 5   Number_deaths        170140 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 9.1+ MB


In [38]:
good_dates.head()

,extract_date,specimen_date,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
0,04/29/2020,2020-04-17,9979,3386,527,96
2,04/29/2020,2020-03-05,63,5,3,1
3,04/29/2020,2020-04-09,9019,4803,1253,386
4,04/29/2020,2020-04-03,9389,5523,1688,582
5,04/29/2020,2020-03-25,7457,4803,1395,429


In [39]:
good_dates.describe()

,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
count,170140.000000,170140.000000,170140.000000,170140.000000
mean,11534.869261,1478.331321,243.238010,76.550829
std,6167.830406,1365.142438,396.333149,162.298195
min,2.000000,0.000000,0.000000,0.000000
25%,7114.000000,434.000000,45.000000,5.000000
50%,11246.000000,1194.000000,120.000000,22.000000
75%,16569.000000,2025.000000,205.000000,45.000000
max,24727.000000,6852.000000,2016.000000,923.000000


In [40]:
high = good_dates[good_dates['Number_tested'] >= 1000]

In [41]:
high.head()

,extract_date,specimen_date,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
0,04/29/2020,2020-04-17,9979,3386,527,96
3,04/29/2020,2020-04-09,9019,4803,1253,386
4,04/29/2020,2020-04-03,9389,5523,1688,582
5,04/29/2020,2020-03-25,7457,4803,1395,429
6,04/29/2020,2020-03-21,4713,2663,757,231


In [42]:
high.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155507 entries, 0 to 176147
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   extract_date         155507 non-null  object        
 1   specimen_date        155507 non-null  datetime64[ns]
 2   Number_tested        155507 non-null  int64         
 3   Number_confirmed     155507 non-null  int64         
 4   Number_hospitalized  155507 non-null  int64         
 5   Number_deaths        155507 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 8.3+ MB


In [43]:
high.describe()

,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
count,155507.000000,155507.000000,155507.000000,155507.000000
mean,12612.174828,1615.444964,265.318802,83.500987
std,5303.235676,1349.081515,407.625626,168.087123
min,1000.000000,0.000000,0.000000,0.000000
25%,8057.000000,603.000000,64.000000,9.000000
50%,12154.000000,1322.000000,131.000000,25.000000
75%,17023.000000,2186.000000,218.000000,49.000000
max,24727.000000,6852.000000,2016.000000,923.000000


In [44]:
low = good_dates[good_dates['Number_tested'] <= 10_000]

In [45]:
low.head()

,extract_date,specimen_date,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
0,04/29/2020,2020-04-17,9979,3386,527,96
2,04/29/2020,2020-03-05,63,5,3,1
3,04/29/2020,2020-04-09,9019,4803,1253,386
4,04/29/2020,2020-04-03,9389,5523,1688,582
5,04/29/2020,2020-03-25,7457,4803,1395,429


In [46]:
low.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75127 entries, 0 to 176145
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   extract_date         75127 non-null  object        
 1   specimen_date        75127 non-null  datetime64[ns]
 2   Number_tested        75127 non-null  int64         
 3   Number_confirmed     75127 non-null  int64         
 4   Number_hospitalized  75127 non-null  int64         
 5   Number_deaths        75127 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 4.0+ MB


In [47]:
low.describe()

,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
count,75127.000000,75127.000000,75127.000000,75127.000000
mean,5724.448587,1289.900182,324.505477,114.405953
std,3291.097568,1654.711794,522.810526,208.499506
min,2.000000,0.000000,0.000000,0.000000
25%,4041.000000,117.000000,17.000000,1.000000
50%,6730.000000,623.000000,75.000000,14.000000
75%,8283.000000,1610.000000,296.000000,82.000000
max,10000.000000,6369.000000,2016.000000,866.000000


In [48]:
high.sort_values('specimen_date', inplace = True)

/home/indigon/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [49]:
high.head()

,extract_date,specimen_date,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
154286,08/26/2021,2020-03-12,1580,489,121,43
21041,09/20/2020,2020-03-12,1574,406,103,34
30384,10/30/2020,2020-03-12,1574,406,102,36
13597,08/09/2020,2020-03-12,1569,405,103,33
1909,04/09/2020,2020-03-12,1561,379,82,17


In [50]:
high

,extract_date,specimen_date,Number_tested,Number_confirmed,Number_hospitalized,Number_deaths
154286,08/26/2021,2020-03-12,1580,489,121,43
21041,09/20/2020,2020-03-12,1574,406,103,34
30384,10/30/2020,2020-03-12,1574,406,102,36
13597,08/09/2020,2020-03-12,1569,405,103,33
1909,04/09/2020,2020-03-12,1561,379,82,17
...,...,...,...,...,...,...
174804,09/29/2021,2021-09-26,6310,199,1,0
175913,10/01/2021,2021-09-26,6310,199,10,0
175566,10/01/2021,2021-09-27,5271,101,11,0
175092,09/30/2021,2021-09-27,5271,101,2,0


In [51]:
print(high['specimen_date'])

154286   2020-03-12
21041    2020-03-12
30384    2020-03-12
13597    2020-03-12
1909     2020-03-12
            ...    
174804   2021-09-26
175913   2021-09-26
175566   2021-09-27
175092   2021-09-27
174292   2021-09-27
Name: specimen_date, Length: 155507, dtype: datetime64[ns]
